In [1]:
import fingertips_py as ftp
from json import loads
import pandas as pd
from urllib.request import urlopen

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

- A unique data set is ['Indicator Name', 'Indicator ID', 'Sex', 'Age', 'Time period', 'AreaTypeId]

- Need to create template table of indictor + area codes so that shape shows in QS

- The indicator name needs to be name + id

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

- time period
- sex
- age

# time period, sex and age available for each indicator

# Area types available for each indicator

In [3]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

print(raw_available_indicator_at_area['AreaTypeId'].unique()
      ,len(raw_available_indicator_at_area['AreaTypeId'].unique()),
      len(raw_available_indicator_at_area),
      raw_available_indicator_at_area['IndicatorId'].nunique()
    #   ,
    #   available_indicator_at_area['IndicatorId'].unique()
)

[402 401 102 101  15   6 221 220 204 167 166  66   7 302 219 152 301 223
 202 165 154 201  46  10 126 104 119 203 118 205 112 129 222   8   3  41] 36 8947 1698


In [4]:
raw_available_indicator_at_area.columns

Index(['IndicatorId', 'AreaTypeId'], dtype='object')

In [5]:
raw_available_indicator_at_area[raw_available_indicator_at_area['IndicatorId']==90813]

,IndicatorId,AreaTypeId
2575,90813,402
2576,90813,302
2577,90813,202
2578,90813,154
2579,90813,102
2580,90813,15
2581,90813,6


# Indicator metadata
To get the name of each indicator

In [6]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')

In [7]:
indicator_metadata['Indicator ID'].nunique()

1408

In [65]:
indicator_metadata.head()

,Indicator ID,Indicator,Definition,Rationale,Data source,Indicator source,Methodology,Standard population/values,Confidence interval details,Source of numerator,Definition of numerator,Source of denominator,Definition of denominator,Disclosure control,Caveats,Copyright,Data re-use,Links,Indicator number,Notes,Frequency,Rounding,Indicator Content,Specific rationale,Impact of COVID-19,Unit,Value type,Year type,Polarity,Date updated
0,108,Under 75 mortality rate from all causes,Directly age-standardised mortality rate for a...,Premature mortality is a good high-level indic...,Office for Health Improvement and Disparities ...,Office for Health Improvement and Disparities,Numerator data for each age band are divided b...,2013 European Standard Population,NaN,Office for National Statistics: Annual Mortali...,Number of deaths registered in the respective ...,ONS 2021 Census based mid year population esti...,"Population for people under 75 years of age, a...",The Isles of Scilly and City of London data ha...,Please note data may not match published ONS f...,Office for Health Improvement and Disparities,The data may be reused referencing Office for ...,NaN,NaN,As the latest data point for this indicator co...,The data will be updated annually,NaN,Directly standardised mortality rate for morta...,NaN,NaN,"per 100,000",Directly standardised rate,Calendar,RAG - Low is good,14/02/2023
1,113,Smoking attributable mortality (old method),"Deaths attributable to smoking, directly age s...",Smoking remains the biggest single cause of pr...,"ONS mortality file, ONS LSOA single year of ag...",Further detail on the methodology can be found...,NaN,European Standard Population 2013,NaN,ONS: Annual birth and death extracts Prevalen...,Smoking attributable deaths are the sums over ...,ONS,ONS mid-year population estimates,Where the observed total number of deaths is l...,NaN,NaN,NaN,NaN,NaN,All data points from 2007-09 to 2011-13 were r...,Annual,NaN,NaN,NaN,NaN,"per 100,000",Directly standardised rate,Calendar,RAG - Low is good,07/10/2019
2,114,QOF Total List Size,Total number of patients registered with the p...,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",For the current year: https://digital.nhs.uk/...,NaN,NaN,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",NaN,"Quality and Outcomes Framework (QOF), NHS Digital",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Count,Financial,Not applicable,27/10/2022
3,200,Learning disability: QOF prevalence (all ages),The percentage of patients with learning disab...,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",For the current year: https://digital.nhs.uk/...,numerator/denominator * 100,NaN,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",Total number of patients with learning disabil...,"Quality and Outcomes Framework (QOF), NHS Digital",Total number of patients registered with the p...,NaN,NaN,NaN,NaN,NaN,NaN,"Where local authority values are presented, th...",NaN,NaN,NaN,NaN,"For 2020/21 QOF data, https://digital.nhs.uk/...",%,Proportion,Financial,BOB - Blue orange blue,27/10/2022
4,212,Stroke: QOF prevalence (all ages),The percentage of patients with stroke or tran...,Stroke is the third most common cause of death...,"Quality and Outcomes Framework (QOF), NHS Digital",For the current year: https://digital.nhs.uk/...,numerator/denominator * 100,NaN,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",Patients with stroke or transient ischaemic at...,"Quality and Outcomes Framework (QOF), NHS Digital",Total practice list size.,NaN,NaN,NaN,NaN,NaN,NaN,"Where local authority values are presented, th...",NaN,NaN,NaN,NaN,"For 2020/21 QOF data, https://digital.nhs.uk/...",%,Proportion,Financial,BOB - Blue orange blue,27/10/2022


In [64]:
indicator_metadata.columns.str.lower().str.replace(' ', '_')

Index(['indicator_id', 'indicator', 'definition', 'rationale', 'data_source',
       'indicator_source', 'methodology', 'standard_population/values',
       'confidence_interval_details', 'source_of_numerator',
       'definition_of_numerator', 'source_of_denominator',
       'definition_of_denominator', 'disclosure_control', 'caveats',
       'copyright', 'data_re-use', 'links', 'indicator_number', 'notes',
       'frequency', 'rounding', 'indicator_content', 'specific_rationale',
       'impact_of_covid-19', 'unit', 'value_type', 'year_type', 'polarity',
       'date_updated'],
      dtype='object')

# Indicator sex, age and timeperiod available

This doesn't actually bring through all of those available in the data. It brings through the settings for the website and what is shown on the website

In [8]:
def df_nested_json(url): 
    with urlopen(url) as response:
        data_str = response.read().decode("utf-8")
        data_json = loads(data_str[1:-1])
    return pd.json_normalize(data_json)

In [9]:
data = df_nested_json('https://fingertips.phe.org.uk/api/grouproot_summaries/by_indicator_id?indicator_ids=90813')

In [10]:
data

,IndicatorName,IID,GroupId,StateSex,StateAge,StateYearRange,TimePeriod,YearRange,Sex.Id,Sex.Name,Age.Id,Age.Name,Age.Sequence,Category.Id,Category.CategoryTypeId,Category.Name,Category.ShortName,DateChanges.HasDataChangedRecently,DateChanges.DateOfLastChange,DataQualityNote.Id,DataQualityNote.Text,DataQualityNote.Comment,Unit.Id,Unit.Value,Unit.Label,ValueType.Id,ValueType.Name
0,Hospital admissions as a result of self-harm (...,90813,1938132923,False,False,False,2021/22,1,4,Persons,305,10-24 yrs,17.0,-1,-1,Undefined,Undefined,False,07 Mar 2023,0,None,None,3,100000.0,"per 100,000",1,Directly standardised rate


# Area Ids metadata
To get the name of the area

In [11]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

In [12]:
area_ref.columns

Index(['Id', 'Name', 'Short', 'Class', 'Sequence', 'CanBeDisplayedOnMap'], dtype='object')

In [13]:
area_ref[area_ref['Class']=='ccg-composite']

,Id,Name,Short,Class,Sequence,CanBeDisplayedOnMap
51,152,CCGs (2017/18),CCGs (2017/18),ccg-composite,2017,True
52,154,CCGs (2018/19),CCGs (2018/19),ccg-composite,2018,True
62,165,CCGs (2019/20),CCGs (2019/20),ccg-composite,2019,True
63,166,CCGs (2020/21),CCGs (2020/21),ccg-composite,2020,True
64,167,CCGs (2021/22),CCGs (from Apr 2021),ccg-composite,2021,True


## hierarchy

In [14]:
area_hierarchy = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/15RhWWsHPPMLWoxR5sJcpK-vraRkidRY8jsAb_Y_5GwI/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
area_hierarchy = area_hierarchy[['AreaTypeId', 'map_position', 'hierarchy']].dropna()

# Area types available for each indicator with extra metadata

## prepare full table

In [15]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata[['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

## create views

In [16]:
# indicators_per_area = (pd.DataFrame(available_indicator_at_area.groupby(['Name', 'Short', 'Class', 'Sequence','CanBeDisplayedOnMap'])['_merge'].count())
#                        .reset_index()
#                        .sort_values(['Class', 'Sequence'], ascending=False)
#                        .rename({'_merge': 'indicator_count'}, axis=1)
#                        )

In [17]:
# indicator_at_area_pivot = available_indicator_at_area[['Indicator', 'Name', 'Id']].pivot_table(index='Indicator', columns='Name', aggfunc='count')

## use hierarchy to choose indicators and areas

In [18]:
full_area_indicator_hierarchy = pd.merge(left=area_hierarchy, right=raw_available_indicator_at_area,
         left_on='AreaTypeId', right_on='AreaTypeId',
         how='inner')

In [19]:
left = full_area_indicator_hierarchy[full_area_indicator_hierarchy['map_position']=='left']
left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')
keep_left = left[left['rank']==1]


C:\Users\medssam\AppData\Local\Temp\ipykernel_11428\2219602755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')


In [20]:
keep_left['rank'].unique()

array([1.])

In [21]:
def find_first_hierarchy(data, map_position):
    data = data[data['map_position']==map_position].loc[:, ['IndicatorId','hierarchy','AreaTypeId', 'map_position']]
    data['rank'] = data.groupby('IndicatorId')['hierarchy'].rank(method='dense')
    return data[data['rank']==1].drop('rank', axis=1)

In [22]:
get_indicator_area = pd.concat([find_first_hierarchy(full_area_indicator_hierarchy, 'left'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'middle'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'right')]
)

# Retrieve indicator data

## filter by those that only have all 3 as a test case

In [23]:
number_of_maps_returned = get_indicator_area.groupby('IndicatorId')['map_position'].count()
ind_ids_keep_3_maps = number_of_maps_returned[number_of_maps_returned==3].index.values

In [24]:
get_ind_area_combos = get_indicator_area[get_indicator_area['IndicatorId'].isin(ind_ids_keep_3_maps)]

## get data

In [26]:
get_ind_area_combos['AreaTypeId'].unique()

array([221, 220, 219,   6,  66, 167, 166, 165, 154, 401, 402, 302, 202,
       102], dtype=int64)

In [27]:
get_ind_area_combos.groupby('AreaTypeId').count()

,IndicatorId,hierarchy,map_position
AreaTypeId,,,
6,45,45,45
66,34,34,34
102,12,12,12
154,15,15,15
165,13,13,13
166,14,14,14
167,61,61,61
202,6,6,6
219,19,19,19


In [28]:
def return_ind_list_for_area_type(data, AreaTypeId):
    return data[data['AreaTypeId']==AreaTypeId]['IndicatorId'].values

In [29]:
return_ind_list_for_area_type(get_ind_area_combos, 221)

array([  200,   212,   219,   241,   247,   253,   258,   273,   848,
       20601, 20602, 90316, 90317, 90319, 90320, 90321, 90323, 90443,
       90581, 90646, 90933, 91215, 91216, 91269, 91280, 91547, 92443,
       92445, 92464, 92465, 92534, 92538, 92588, 93194, 93195, 93382,
       93553], dtype=int64)

In [30]:
def get_data(data, area_type_id):
    indicator_list = return_ind_list_for_area_type(data, area_type_id)
    
    replace_comma = '%2C'
    ids_as_str = replace_comma.join([str(i) for i in indicator_list])
    ids_as_str
    data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
                                                    area_type_id=area_type_id,
                                                    # parent_area_type_id=15,
                                                    # profile_id=None,
                                                    include_sortable_time_periods=True,
                                                    is_test=False)
    
    return data
    
    # data.to_csv(str(area_type_id)+'.csv')

In [31]:
# for area_id in get_ind_area_combos['AreaTypeId'].unique():
#     get_data(get_ind_area_combos, area_id)

In [32]:
pd.read_json('https://fingertips.phe.org.uk/api/indicator_metadata/by_indicator_id?indicator_ids=1204&include_definition=yes&include_system_content=yes')

,1204
AlwaysShowAgeWithIndicatorName,False
AlwaysShowSexWithIndicatorName,False
ConfidenceIntervalMethod,"{'Name': 'Dobson & Byar's methods', 'Descripti..."
DataChange,"{'LastUploadedAt': '2023-02-21T12:30:26', 'Las..."
DataQualityNote,"{'Id': 0, 'Text': 'None', 'Comment': None}"
Descriptive,{'Name': 'Mortality rate from chronic obstruct...
IID,1204
InequalitiesExcludedCategoryTypeIds,[]
LatestChangeTimestampOverride,2021-11-16T00:00:00
ShouldCountBeDisplayed,True


In [33]:
# data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
#  113,
#  114,
#  200,
#  212],
#                                                                area_type_id=7,
#                                                                parent_area_type_id=15,
#                                                                filter_by_area_codes=None,
#                                                                is_test=False)

In [34]:
# # metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# # metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# # metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# # metadata_domains = ftp.metadata.get_domains_in_profile(18)
# metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
# metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [35]:
# metadata_all = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/all?include_definition=yes&include_system_content=yes')

# Analysis of nulls

In [36]:
area_ref[area_ref['Class'].str.contains('ccg-composite')]

,Id,Name,Short,Class,Sequence,CanBeDisplayedOnMap
51,152,CCGs (2017/18),CCGs (2017/18),ccg-composite,2017,True
52,154,CCGs (2018/19),CCGs (2018/19),ccg-composite,2018,True
62,165,CCGs (2019/20),CCGs (2019/20),ccg-composite,2019,True
63,166,CCGs (2020/21),CCGs (2020/21),ccg-composite,2020,True
64,167,CCGs (2021/22),CCGs (from Apr 2021),ccg-composite,2021,True


In [37]:
analyse_nulls_data = get_data(get_ind_area_combos, 165)
print(analyse_nulls_data['Value'].isna().sum())
# remove null Values
analyse_nulls_data = analyse_nulls_data[~analyse_nulls_data['Value'].isna()]
print(analyse_nulls_data['Value'].isna().sum())

# remove England
print(analyse_nulls_data['Area Name'].nunique())
analyse_nulls_data = analyse_nulls_data[analyse_nulls_data['Area Name']!='England']
print(analyse_nulls_data['Area Name'].nunique())


6455
0
192
191


In [38]:
analyse_nulls_data['Indicator Name'].nunique()

10

In [39]:
analyse_nulls_data.columns

Index(['Indicator ID', 'Indicator Name', 'Parent Code', 'Parent Name',
       'Area Code', 'Area Name', 'Area Type', 'Sex', 'Age', 'Category Type',
       'Category', 'Time period', 'Value', 'Lower CI 95.0 limit',
       'Upper CI 95.0 limit', 'Lower CI 99.8 limit', 'Upper CI 99.8 limit',
       'Count', 'Denominator', 'Value note', 'Recent Trend',
       'Compared to England value or percentiles', 'Compared to percentiles',
       'Time period Sortable', 'New data', 'Compared to goal',
       'Time period range'],
      dtype='object')

In [41]:
# analyse_nulls_data.groupby(['Indicator Name', 'Indicator ID', 'Sex', 'Age', 'Time period'])['Area Code'].nunique()

# Analysis of indicator ids and names

Some indicator ids have the same name - do they have the same metadata?

No, they have different metadata so a unique indicator would be (indicator id + name)

In [51]:
indicator_ids_per_name = pd.DataFrame(indicator_metadata.groupby('Indicator')['Indicator ID'].nunique()).reset_index()


In [54]:
indicators_with_same_name = indicator_ids_per_name[indicator_ids_per_name['Indicator ID']>1]

In [55]:
indicators_with_same_name.head()

,Indicator,Indicator ID
32,A&E attendances (0-4 years),3
33,A&E attendances (<18),2
34,A&E attendances (under 1 year),2
63,Admission episodes for alcohol-specific condit...,2
70,Admissions for asthma for children aged 0 to 9,2


In [59]:
indicator_name = indicators_with_same_name['Indicator'].iloc[0]
indicator_metadata[indicator_metadata['Indicator']==indicator_name]

,Indicator ID,Indicator,Definition,Rationale,Data source,Indicator source,Methodology,Standard population/values,Confidence interval details,Source of numerator,Definition of numerator,Source of denominator,Definition of denominator,Disclosure control,Caveats,Copyright,Data re-use,Links,Indicator number,Notes,Frequency,Rounding,Indicator Content,Specific rationale,Impact of COVID-19,Unit,Value type,Year type,Polarity,Date updated
245,90809,A&E attendances (0-4 years),"A&E attendance rate per 1,000 population aged ...",A&E attendances in children aged under five ye...,Hospital Episode Statistics (HES) Copyright © ...,"Children, Families and Healthy Ageing. Office ...","Crude rate per 1,000 population",NaN,95% confidence limits have been calculated usi...,Hospital Episode Statistics (HES),A&E attendances for all children aged 0-4 years.,Office for National Statistics,Mid-year population estimates: Single year of ...,NaN,"From 2018/19 onwards, all sub-national counts ...",Copyright © NHS Digital. All rights reserved.\...,Reused with the permission of NHS Digital. All...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020/21 HES A&E data is not available. Some da...,"per 1,000",Crude rate,Financial,RAG - Low is good,21/02/2022
733,92757,A&E attendances (0-4 years),"A&E attendance rate per 1,000 registered popul...",A&E attendances in children aged under five ye...,"OHID based on NHS Digital, Hospital Episode St...",NaN,"Crude rate per 1,000 population",NaN,95% confidence limits have been calculated usi...,"NHS Digital, Hospital Episode Statistics (HES)",A&E attendances for children or young people o...,NHS Digital,CCG populations are aggregated from NHS Digita...,NaN,The A&E data set is a developing data set whic...,Copyright © NHS Digital. All rights reserved.\...,Re‐used with the permission of NHS Digital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"per 1,000",Crude rate,Financial,RAG - Low is good,29/11/2022
1384,93930,A&E attendances (0-4 years),"A&E attendance rate per 1,000 population aged ...",A&E attendances in children aged under five ye...,"Office for Health Improvement and Disparities,...",NaN,"Crude rate per 1,000 population",NaN,95% confidence limits have been calculated usi...,"Emergency Care Data Set (ECDS), part of Hospit...",A&E attendances for all children aged 0-4 year...,Office for National Statistics,Mid-year population estimates: Single year of ...,NaN,"From 2018/19 onwards, all sub-national counts ...",Copyright © NHS Digital. All rights reserved.\...,Reused with the permission of NHS Digital. All...,NaN,NaN,This indicator replaces 90809 with the same na...,NaN,NaN,NaN,NaN,NaN,"per 1,000",Crude rate,Financial,RAG - Low is good,21/02/2023
